In [2]:
#dataset
import json
import pandas as pd
import re
import jieba 
import jieba.analyse
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler
import scipy.stats as stats

In [3]:
# Restore to default settings
pd.reset_option('display.max_colwidth')

In [15]:
# Read the JSON file into a pandas DataFrame
df_depressed = pd.read_json('depressed.json')


In [16]:
# import the normal user dataset
# with open('normal.json', 'r') as file:
#     # Load the JSON data
#     normal = json.load(file)

df_normal = pd.read_json('normal.json')
# normal user， label=0, has errors, need to update
df_normal['label'] = 0

In [17]:
df_depressed.head(5)

,label,nickname,gender,profile,birthday,num_of_follower,num_of_following,all_tweet_count,original_tweet_count,repost_tweet_count,tweets
0,1,迷失路径,男,此人严重丧，不适绕行，谢谢,1993-12-10,1,0,10,10,0,"[{'tweet_content': '有点累想休息了', 'posting_time': ..."
1,1,九七97吖,女,无,1997-06-07,173,76,219,79,70,"[{'tweet_content': '轉發微博', 'posting_time': '20..."
2,1,姑先十八-,女,喜提呼伦贝尔大草原五个月。,无,22,205,6188,24,5,[{'tweet_content': '抑郁症的普及它好就好在 以前人抑郁的时候想死 现在人...
3,1,大邱百香果,女,无,无,2,9,62,17,6,"[{'tweet_content': '', 'posting_time': '2017-0..."
4,1,落墨暮烟,女,无,2001-09-25,1,27,6,6,0,[{'tweet_content': '所有人都问我你没事吧你还好吧 我笑着一一回答我很好啊...


In [12]:
# read csv 
# df_normal = pd.read_csv('df_normal.csv')
# df_depressed = pd.read_csv('df_depressed.csv')

In [18]:
#insert ID
df_depressed = df_depressed.reset_index().rename(columns={'index': 'ID'})
df_depressed['ID'] += 1



In [19]:
df_normal = df_normal.reset_index().rename(columns={'index': 'ID'})
df_normal['ID'] += 1

In [20]:
df_depressed.head(5)

,ID,label,nickname,gender,profile,birthday,num_of_follower,num_of_following,all_tweet_count,original_tweet_count,repost_tweet_count,tweets
0,1,1,迷失路径,男,此人严重丧，不适绕行，谢谢,1993-12-10,1,0,10,10,0,"[{'tweet_content': '有点累想休息了', 'posting_time': ..."
1,2,1,九七97吖,女,无,1997-06-07,173,76,219,79,70,"[{'tweet_content': '轉發微博', 'posting_time': '20..."
2,3,1,姑先十八-,女,喜提呼伦贝尔大草原五个月。,无,22,205,6188,24,5,[{'tweet_content': '抑郁症的普及它好就好在 以前人抑郁的时候想死 现在人...
3,4,1,大邱百香果,女,无,无,2,9,62,17,6,"[{'tweet_content': '', 'posting_time': '2017-0..."
4,5,1,落墨暮烟,女,无,2001-09-25,1,27,6,6,0,[{'tweet_content': '所有人都问我你没事吧你还好吧 我笑着一一回答我很好啊...


In [21]:
df_normal.head(5)

,ID,label,nickname,gender,profile,birthday,num_of_follower,num_of_following,all_tweet_count,original_tweet_count,repost_tweet_count,tweets
0,1,0,汤圆s点点滴滴,女,小日常,无,3,23,44,43,1,"[{'tweet_content': '酸汤肥牛底下还有你家的豆皮', 'posting_t..."
1,2,0,心无挂碍wxq,女,传播正能量，广结善缘。每天懂一点佛学知识，感悟人生，净化身心，弘扬佛法，传播中国传统文化，便...,2000-08-02,581,968,2877,98,0,"[{'tweet_content': '无', 'posting_time': '2020-..."
2,3,0,透心凉A兔子,女,无,1998-04-10,1363,241,1326,97,1,[{'tweet_content': '单身狗不能接受的路过 还有谁不知道里的求婚场面今晚的...
3,4,0,孔阿君,女,无,无,5,64,5,5,0,"[{'tweet_content': '君君发了一条微博', 'posting_time':..."
4,5,0,路路090909,男,所谓活着的人，就是不断挑战的人，不断攀登命运峻峰的人。,无,141,1480,1883,62,38,"[{'tweet_content': '我不喜欢这世界我只喜欢你', 'posting_ti..."


In [22]:
#data balance
df_merged1 = pd.concat((df_depressed, df_normal))

In [23]:
# before balance1
grouped_label = df_merged1.groupby(['label', ]).size().reset_index(name='count')

In [24]:
grouped_label

,label,count
0,0,22245
1,1,10325


In [25]:
grouped_gender = df_merged1.groupby(['gender', ]).size().reset_index(name='count')

In [26]:
df_depressed['gender'].value_counts()

女    7726
男    2599
Name: gender, dtype: int64

In [27]:
df_normal['gender'].value_counts()

女    16920
男     5325
Name: gender, dtype: int64

In [28]:
grouped_gender

,gender,count
0,女,24646
1,男,7924


In [34]:

# Select random 5000 rows where label = 0 and gender = "女"

df_female = df_merged1[(df_merged1['label'] == 0) & (df_merged1['gender'] == '女')].sample(n=5000, random_state=42)

# Select all rows where label = 0 and gender = "男"
df_male = df_merged1[(df_merged1['label'] == 0) & (df_merged1['gender'] == '男')]

# Select all rows where label = 1
df_label_1 = df_merged1[df_merged1['label'] == 1]

# Merge all three parts of rows
df_final_balanced_all = pd.concat([df_female, df_male, df_label_1])

In [35]:
# Save final data to CSV file
df_final_balanced_all.to_csv('df_merged_balanced_all_columns.csv', index=False)

In [36]:
# data balance method2

df_final_balanced_all = pd.read_csv('df_merged_balanced_all_columns.csv')

In [37]:
# after data balance1, label
grouped_label = df_final_balanced_all.groupby(['label', ]).size().reset_index(name='count')

In [38]:
grouped_label

,label,count
0,0,10325
1,1,10325


In [39]:
# after data balance1,gender
grouped_gender = df_final_balanced_all.groupby(['gender', ]).size().reset_index(name='count')

In [40]:
grouped_gender

,gender,count
0,女,12726
1,男,7924
